<table>
<tr>
<th>Name</th>
<th>Section</th>
<th>B.N</th>
<th>ID</th>
</tr>
<tr>
<th>Beshoy Morad Atya</th>
<th>1</th>
<th>19</th>
<th>9202405</th>
</tr>
<tr>
<th>Peter Atef Fathi</th>
<th>1</th>
<th>18</th>
<th>9202395</th>
</tr>
</table>

### Imports

In [1]:
%pip install graphviz

In [2]:
import json
import graphviz
import re
from enum import Enum, auto
from dataclasses import dataclass

### Parser

In [3]:
EPSILON = 'epsilon'

class TokenType(Enum):
    OR = auto()                 # |
    STAR = auto()               # *
    PLUS = auto()               # +
    QUESTION_MARK = auto()      # ?
    OPEN_PAREN = auto()         # (
    CLOSED_PAREN = auto()       # )
    OPEN_SQ_BRACKET = auto()    # [
    CLOSED_SQ_BRACKET = auto()  # ]
    DASH = auto()               # -
    LITERAL = auto()            # a-z A-Z 0-9
    DOT = auto()                # .

@dataclass
class Token:
    token_type: TokenType
    string: str

class AstNode:
    pass

@dataclass
class OrAstNode(AstNode):
    left: AstNode
    right: AstNode

@dataclass
class SeqAstNode(AstNode):
    left: AstNode
    right: AstNode

@dataclass
class StarAstNode(AstNode):
    left: AstNode

@dataclass
class PlusAstNode(AstNode):
    left: AstNode

@dataclass
class QuestionMarkAstNode(AstNode):
    left: AstNode

@dataclass
class LiteralCharacterAstNode(AstNode):
    char: str


def parse_regex(tokens, current_token):
    """
    regex-expression -> or-expression
    """
    return parse_or(tokens, current_token)

def parse_or(tokens, current_token):
    """
    or-expression -> sequence-expression (OR sequence-expression)*
    """
    # Get the left sequence expression
    left, current_token = parse_sequence(tokens, current_token)

    # Check if there are no more tokens
    if current_token >= len(tokens):
        return left, current_token

    # Final AST node
    or_expression = left

    # Loop through the rest of the tokens
    while current_token < len(tokens) and tokens[current_token].token_type == TokenType.OR:
        current_token += 1

        # Get the right sequence expression
        right, current_token = parse_sequence(tokens, current_token)

        or_expression = OrAstNode(left=or_expression, right=right)

    return or_expression, current_token

def parse_sequence(tokens, current_token):
    """
    sequence-expression -> quantified-expression (quantified-expression)*
    """
    # Get the left quantified expression
    left, current_token = parse_quantified(tokens, current_token)

    # Check if there are no more tokens
    if current_token >= len(tokens):
        return left, current_token

    # Final AST node
    sequence_expression = left

    # Loop through the rest of the tokens
    while current_token < len(tokens) \
        and tokens[current_token].token_type != TokenType.OR \
        and tokens[current_token].token_type != TokenType.CLOSED_PAREN:

        right, current_token = parse_quantified(tokens, current_token)

        sequence_expression = SeqAstNode(left=sequence_expression, right=right)

    return sequence_expression, current_token

def parse_quantified(tokens, current_token):
    """
    quantified-expression -> base-expression (STAR | PLUS | QUESTION_MARK)?
    """
    # Get the left base expression
    left, current_token = parse_base(tokens, current_token)

    # Check if there are no more tokens
    if current_token >= len(tokens):
        return left, current_token

    if tokens[current_token].token_type == TokenType.STAR:
        current_token += 1
        return StarAstNode(left=left), current_token

    if tokens[current_token].token_type == TokenType.PLUS:
        current_token += 1
        return PlusAstNode(left=left), current_token

    if tokens[current_token].token_type == TokenType.QUESTION_MARK:
        current_token += 1
        return QuestionMarkAstNode(left=left), current_token

    return left, current_token

def parse_base(tokens, current_token):
    """
    base-expression -> LITERAL
                    | DOT
                    | OPEN_PAREN regex-expression CLOSED_PAREN
                    | OPEN_SQ_BRACKET sq-bracket-content CLOSED_SQ_BRACKET
    """
    # Check if there are no more tokens
    if current_token >= len(tokens):
        raise Exception("No more tokens to parse")

    token = tokens[current_token]
    current_token += 1

    if token.token_type == TokenType.LITERAL:
        return LiteralCharacterAstNode(char=token.string), current_token

    if token.token_type == TokenType.DOT:
        # Epsilon because dot match any character
        return LiteralCharacterAstNode(char='.'), current_token

    if token.token_type == TokenType.OPEN_PAREN:
        expression, current_token = parse_regex(tokens, current_token)

        if current_token >= len(tokens):
            raise Exception("No more tokens to parse")

        if tokens[current_token].token_type != TokenType.CLOSED_PAREN:
            raise Exception("Expected closed parenthesis")

        current_token += 1
        return expression, current_token

    if token.token_type == TokenType.OPEN_SQ_BRACKET:
        sq_bracket, current_token = parse_sq_bracket_content(tokens, current_token)

        if current_token >= len(tokens):
            raise Exception("No more tokens to parse")

        if tokens[current_token].token_type != TokenType.CLOSED_SQ_BRACKET:
            raise Exception("Expected square bracket")

        current_token += 1
        return sq_bracket, current_token

def parse_sq_bracket_content(tokens, current_token):
    """
    sq-bracket-content -> LITERAL
                        | LITERAL DASH LITERAL
    """
    # Check if there are no more tokens
    if current_token >= len(tokens):
        raise Exception("No more tokens to parse")

    chars = []
    is_dash_reached = False

    while current_token < len(tokens) \
        and tokens[current_token].token_type != TokenType.CLOSED_SQ_BRACKET:

        if tokens[current_token].token_type == TokenType.DASH:
            is_dash_reached = True
        elif is_dash_reached:
            if len(chars) == 0:
                raise Exception("Expected at least one character")

            range_start = chars.pop()
            range_end = tokens[current_token].string
            chars.append((range_start, range_end))
            is_dash_reached = False
        else:
            chars.append(tokens[current_token].string)

        current_token += 1

    if is_dash_reached:
        raise Exception("Wrong range format")

    # Handle the cases like [a-z b 0-5] should be like (a-z | b | 0-5)
    expression = None
    for char in chars:
        if isinstance(char, tuple):
            if expression is None:
                expression = LiteralCharacterAstNode(char=f"{char[0]}-{char[1]}")
            else:
                expression = OrAstNode(left=expression, right=LiteralCharacterAstNode(char=f"{char[0]}-{char[1]}"))
        else:
            if expression is None:
                expression = LiteralCharacterAstNode(char=char)
            else:
                expression = OrAstNode(left=expression, right=LiteralCharacterAstNode(char=char))

    return expression, current_token

### Json Serializer

In [4]:
class JsonSerialize:
    def __init__(self):
        pass

    def nfa_json_serialize(self, nfa):
        """_summary_

        Args:
            nfa (NFA): The NFA data object

        Returns:
            dict: dictionary of the json data
        """
        json_data = {}

        address_to_name = {}
        from_state = {}

        for _, state in enumerate(nfa.states):
            name = f"S{str(nfa.states.index(state))}"
            address_to_name[state] = name
            from_state[name] = []

        for transition in nfa.transitions:
            # Get all the states that the transition goes to
            from_state[address_to_name[transition.from_]].append(
                {"to": address_to_name[transition.to_], "char": transition.characters}
            )

        # Extracting starting state
        json_data["startingState"] = address_to_name[nfa.start]

        for _, state in enumerate(from_state):
            epsilons = 1
            json_data[state] = {
                "isTerminatingState": state == address_to_name[nfa.accept],
            }

            transitions = {}
            for transition in from_state[state]:
                if transition["char"] == "epsilon":
                    transitions[f"epsilon{epsilons}"] = transition["to"]
                    epsilons += 1
                else:
                    transitions[transition["char"]] = transition["to"]

            # Check if transitions is empty
            if len(transitions) > 0:
                json_data[state].update(transitions)

        return json_data

    def dfa_json_serialize(self, dfa):
        """_summary_
        This function is used to serialize the DFA object to JSON format
        Args:
            dfa (DFA): the DFA data object
        Returns:
            dict: dictionary of the json data
        """
        json_data = {}
        # set the starting state in the json data
        json_data["startingState"] = dfa.start.name

        # loop over the states in the DFA
        for _, state in enumerate(dfa.states):
            # loop over the transitions in the state
            json_data[state.name] = {
                "isTerminatingState": state in dfa.accept,
            }
            for transition in dfa.transitions:
                if transition.from_ == state:
                    json_data[state.name][
                        "".join(list(transition.characters))
                    ] = transition.to_.name
        return json_data


### Graph Visualizer

In [5]:
class GraphVisualize:
    def __init__(self):
        pass

    def graph_visualize(self, name="graph", json_data=None):
        """_summary_
        Args:
            name (str, optional): name of the file .gv. Defaults to "graph_visualization.gv".
            json_data (dict): Dictionary of the json data. Defaults to None.

        Returns:
            None: no return value
        """
        try:
            if json_data is None:
                print("Error: No data to visualize")
                return False
            graph = graphviz.Digraph(engine="dot")

            for state, transitions in json_data.items():
                if state == "startingState":
                    graph.edge("Start", transitions)
                    continue

                if transitions.get("isTerminatingState", False):
                    graph.node(state, shape="doublecircle")
                else:
                    graph.node(state, shape="circle")

                for char, next_state in transitions.items():
                    if char == "isTerminatingState":
                        continue
                    if "epsilon" in char:
                        graph.edge(state, next_state, label="epsilon")
                    else:
                        graph.edge(state, next_state, label=char)

            graph.render(name, format="png", cleanup=True)
            return True
        except Exception as e:
            print(f"Error: {e}")
            return False


### Regex to NFA

In [6]:
class State:
    pass


@dataclass
class Edge:
    from_: State
    to_: State
    characters: str  # of chars (literals, epsilon)

@dataclass
class NFA:
    start: State
    accept: State

    # All states of the NFA including the start and accept
    states: list  # of State
    transitions: list  # of Edge


class NFA_CLASS:
    def __init__(self, regex):
        if not self.check_regex(regex):
            raise ValueError(f"Invalid regular expression: {regex}")

        self._regex = regex
        self._tokens = []
        self._ast = None
        self._nfa = None
        self._nfa_json = None

        self.tokenize()
        self.parse()
        self.AST_to_NFA()
        self.nfa_to_json()
        self.nfa_visualize()

    def check_regex(self, regex):
        """
        Function used to check if the regex is valid or not
        """
        try:
            re.compile(regex)
        except re.error:
            print(f"Invalid regular expression: {regex}")
            return False
        return True

    def tokenize(self):
        """
        Function used to tokenize the regex, and handle the escape char '\'
        that helps us to escape the special characters in regex | * + ? ( ) [ ] - .
        """
        character_map = {
            "|": TokenType.OR,
            "*": TokenType.STAR,
            "+": TokenType.PLUS,
            "?": TokenType.QUESTION_MARK,
            "(": TokenType.OPEN_PAREN,
            ")": TokenType.CLOSED_PAREN,
            "[": TokenType.OPEN_SQ_BRACKET,
            "]": TokenType.CLOSED_SQ_BRACKET,
            "-": TokenType.DASH,
            ".": TokenType.DOT,
        }
        escape = "\\"

        token_stream = []
        prev_char = None

        for char in self._regex:
            if char == escape:
                prev_char = char
                continue

            if char in character_map and prev_char != escape:
                token_stream.append(Token(character_map[char], char))
            else:
                token_stream.append(Token(TokenType.LITERAL, char))

            prev_char = char

        self._tokens = token_stream

    def parse(self):
        """
        Function used to create the Abstract Syntax Tree (AST) of the regex using the following grammar

        regex-expression        -> or-expression
        or-expression           -> sequence-expression (OR sequence-expression)*
        sequence-expression     -> quantified-expression (quantified-expression)*
        quantified-expression   -> base-expression (STAR | PLUS | QUESTION_MARK)?
        base-expression         -> LITERAL
                                | DOT
                                | OPEN_PAREN regex-expression CLOSED_PAREN
                                | OPEN_SQ_BRACKET sq-bracket-content CLOSED_SQ_BRACKET
        sq-bracket-content      -> LITERAL
                                | LITERAL DASH LITERAL
        """
        expression, _ = parse_regex(self._tokens, 0)
        self._ast = expression

    def construct_nfa(self, node):
        if isinstance(node, OrAstNode):
            return self.construct_or_nfa(node)
        elif isinstance(node, SeqAstNode):
            return self.construct_seq_nfa(node)
        elif isinstance(node, StarAstNode) or isinstance(node, PlusAstNode):
            return self.construct_star_plus_nfa(node)
        elif isinstance(node, QuestionMarkAstNode):
            return self.construct_question_mark_nfa(node)
        elif isinstance(node, LiteralCharacterAstNode):
            return self.construct_literal_class_nfa(node)

    def construct_or_nfa(self, node):
        # If we have regex A|B

        # Get the NFAs representing A and B
        left_nfa = self.construct_nfa(node.left)
        right_nfa = self.construct_nfa(node.right)

        # Create 2 new states, start and accept for the full NFA
        start = State()
        accept = State()

        # Link the start state with both the start of A's NFA and B's NFA with an epsilon transitions
        start_transition_1 = Edge(from_=start, to_=left_nfa.start, characters=EPSILON)
        start_transition_2 = Edge(
            from_=start, to_=right_nfa.start, characters=EPSILON
        )

        # Link the accept state with both the accept of A's NFA and B's NFA with an epsilon transitions
        final_transition_1 = Edge(
            from_=left_nfa.accept, to_=accept, characters=EPSILON
        )
        final_transition_2 = Edge(
            from_=right_nfa.accept, to_=accept, characters=EPSILON
        )

        # Finalize the Or NFA and return it
        states = [*left_nfa.states, *right_nfa.states, start, accept]
        transitions = [
            *left_nfa.transitions,
            *right_nfa.transitions,
            start_transition_1,
            start_transition_2,
            final_transition_1,
            final_transition_2,
        ]
        return NFA(start, accept, states, transitions)

    def construct_seq_nfa(self, node):
        # If we have regex AB

        # Get the NFAs representing A and B
        left_nfa = self.construct_nfa(node.left)
        right_nfa = self.construct_nfa(node.right)

        # Link the two NFAs by connecting the accept state of A to the start state of B
        final_transition = Edge(
            from_=left_nfa.accept, to_=right_nfa.start, characters=EPSILON
        )

        # Finalize the Seq NFA and return it
        states = [*left_nfa.states, *right_nfa.states]
        transitions = [*left_nfa.transitions, *right_nfa.transitions, final_transition]
        return NFA(left_nfa.start, right_nfa.accept, states, transitions)

    def construct_star_plus_nfa(self, node):
        # If we have regex A* or A+
        is_star = isinstance(node, StarAstNode)

        # Get the NFA representing A
        left_nfa = self.construct_nfa(node.left)

        # Create 2 new states, start and accept for the full NFA
        start = State()
        accept = State()

        # Link the new start to the start of A's NFA
        start_transition_1 = Edge(from_=start, to_=left_nfa.start, characters=EPSILON)

        # For star nodes only, it also goes to the accept state directly to represent accepting an empty inputs
        start_transition_2 = Edge(from_=start, to_=accept, characters=EPSILON)

        # Link the accept state with the accept of A's NFA
        final_transition_1 = Edge(
            from_=left_nfa.accept, to_=accept, characters=EPSILON
        )

        # To represent zero|one 'or more' we link the accept state of A with the new start state
        final_transition_2 = Edge(
            from_=left_nfa.accept, to_=start, characters=EPSILON
        )

        # Finalize the star or plus NFA and return it
        states = [*left_nfa.states, start, accept]
        transitions = [
            *left_nfa.transitions,
            start_transition_1,
            final_transition_1,
            final_transition_2,
        ]
        if is_star:
            transitions.append(start_transition_2)

        return NFA(start, accept, states, transitions)

    def construct_question_mark_nfa(self, node):
        # If we have regex A?

        # Get the NFA representing A
        left_nfa = self.construct_nfa(node.left)

        # Create 2 new states, start and accept for the full NFA
        start = State()
        accept = State()

        # Link the new start to the start of A's NFA
        start_transition_1 = Edge(from_=start, to_=left_nfa.start, characters=EPSILON)

        # It also goes to the accept state directly to represent accepting an empty inputs
        start_transition_2 = Edge(from_=start, to_=accept, characters=EPSILON)

        # Link the accept state with the accept of A's NFA
        final_transition_1 = Edge(
            from_=left_nfa.accept, to_=accept, characters=EPSILON
        )

        # Finalize the question mark NFA and return it
        states = [*left_nfa.states, start, accept]
        transitions = [
            *left_nfa.transitions,
            start_transition_1,
            start_transition_2,
            final_transition_1,
        ]

        return NFA(start, accept, states, transitions)

    def construct_literal_class_nfa(self, node):
        # If we have regex 'x' for any character x

        # Create 2 new states, start and accept for the full NFA
        start = State()
        accept = State()

        # Single transition that goes from the starting to the accepting on the relevant characters
        final_transition = Edge(start, accept, node.char)

        # Finalize the literal and character class NFA and return it
        states = [start, accept]
        transitions = [final_transition]

        return NFA(start, accept, states, transitions)

    def AST_to_NFA(self):
        self._nfa = self.construct_nfa(self._ast)

    def nfa_to_json(self):
        """
        This function is used to convert the NFA to a JSON format
        """
        json_serialize = JsonSerialize()
        self._nfa_json = json_serialize.nfa_json_serialize(self._nfa)
        del json_serialize

        # Store the nfa json in a file
        with open("nfa.json", "w") as f:
            json.dump(self._nfa_json, f, indent=4)

    def nfa_visualize(self, path="nfa"):
        graph_visualize = GraphVisualize()
        if graph_visualize.graph_visualize(path, self._nfa_json):
            print(f"Visualization of the NFA is saved in {path}")
        else:
            print("Error: Visualization failed")
        del graph_visualize



### NFA to DFA

In [7]:
@dataclass
class DfaState:
    name: str = None

    def __hash__(self) -> int:
        return hash(self.name)


@dataclass
class DfaEdge:
    from_: DfaState
    to_: DfaState
    characters: set  # of chars (literals, epsilon) and pairs of chars (for ranges)


@dataclass
class DFA:
    start: DfaState
    accept: list  # list of accepting states
    non_accept: list  # list of non accepting states
    # All states of the DFA including the start and accept
    states: list  # of State
    transitions: list  # of Edge


class DFA_CLASS:
    def __init__(self, dfa=None, inputs=[]):
        """_summary_
        class that contains the methods to convert NFA to DFA
        dfa: DFA object
        inputs: list of inputs of the NFA
        """
        self._dfa = dfa
        self.inputs = inputs

    def epsilon_closure(self, nfa, state):
        """_summary_
        This function is used to get the epsilon closure of a given state
        Args:
            nfa (NFA): this a NFA object contains the NFA data
            state (State): this is the state that we want to get the epsilon closure for

        Returns:
            set: closure set of states that are reachable from the given state using epsilon transitions
        """
        closure = set()
        # add the state to its closure set
        closure.add(state)
        for edge in nfa.transitions:
            if edge.from_ == state and EPSILON in edge.characters:
                # add the state that is reachable using epsilon transition to the closure set
                closure.add(edge.to_)
                # call the function recursively to get the epsilon closure of the next state
                closure = closure.union(self.epsilon_closure(nfa, edge.to_))
        return closure

    def move(self, transitions, states, character):
        """_summary_
        This function is used to get the move of a given set of states using a given character
        Args:
            transitions (List): list of transitions of the NFA (list of edges)
            states (set): set of states that we want to get the move for. states list is a subset of nfa.states because we apply the input on only a subset
            character (str): the character that we want to move to

        Returns:
            set: set of states that are reachable from the given states using the given character
        """
        move = set()
        for state in states:
            for edge in transitions:
                if edge.from_ == state and character in edge.characters:
                    move.add(edge.to_)
        return move

    def subset_construction(self, nfa, inputs):
        """_summary_
        This function is used to convert NFA to DFA using the subset construction algorithm
        Args:
            nfa (NFA): this a NFA object contains the NFA data
            inputs (list): list of inputs of the NFA
        """
        # get the epsilon closure of the start state
        start_set = self.epsilon_closure(nfa, nfa.start)
        # get the inputs of the NFA
        self.inputs = inputs
        # list of transitions of the DFA
        transitions = []
        # list of states of the DFA
        states = []
        # add the start state to the list of states of the DFA
        states.append(start_set)
        # list of accepting states of the DFA
        accept = []
        # list of non accepting states of the DFA
        non_accept = []
        # list of states that we need to visit
        to_visit = [start_set]
        # while there are states to visit
        while to_visit:
            # get the first state in the list of states to visit
            current_state = to_visit.pop(0)
            # for each input in the inputs list
            for input_ in self.inputs:
                # get the move of the current state using the input
                move = self.move(nfa.transitions, current_state, input_)
                # get the epsilon closure of the move set
                for state in move:
                    move = move.union(self.epsilon_closure(nfa, state))
                # if the move set is not empty
                if move:
                    # add the move set to the list of states of the DFA
                    if move not in states:
                        states.append(move)
                        to_visit.append(move)
                    # add the transition from the current state to the move set using the input
                    transitions.append(
                        DfaEdge(from_=current_state, to_=move, characters={input_})
                    )

        # we need to convert every set of states to a state object
        for i, state in enumerate(states):
            temp = state
            states[i] = DfaState(name=",".join([state.name for state in state]))
            # loop over the transitions to update the from_ and to_ states
            for transition in transitions:
                if transition.from_ == temp:
                    transition.from_ = states[i]
                if transition.to_ == temp:
                    transition.to_ = states[i]
            # if the state contains an accepting state of the NFA
            if temp.intersection({nfa.accept}):
                # add the state to the list of accepting states of the DFA
                accept.append(states[i])
            else:
                # add the state to the list of non accepting states of the DFA
                non_accept.append(states[i])
        # create a DFA object
        self._dfa = DFA(
            start=states[0],
            accept=accept,
            non_accept=non_accept,
            states=states,
            transitions=transitions,
        )

    def print_dfa(self):
        """
        This function is used to print the DFA object
        """
        print("Start State: ", self._dfa.start.name)
        print("Accepting States: ", [state.name for state in self._dfa.accept])
        print("Non Accepting States: ", [state.name for state in self._dfa.non_accept])
        print("States: ", [state.name for state in self._dfa.states])
        print("Transitions: ")
        for transition in self._dfa.transitions:
            print(
                f"From: {transition.from_.name}, To: {transition.to_.name}, Characters: {transition.characters}"
            )

    def rename_dfa_states(self):
        """
        This function is used to rename the states of the DFA to S0, S1, S2, ...
        """
        for i, state in enumerate(self._dfa.states):
            state.name = f"S{i}"

    def dfa_to_json(self, file_path: str):
        """
        This function is used to convert the DFA to a JSON format (serialize the DFA object)
        Args:
            file_path (str): the path of the file that we want to store the JSON data in
        Returns:
            None: no return value
        """
        json_serialize = JsonSerialize()
        self.dfa_json = json_serialize.dfa_json_serialize(self._dfa)
        del json_serialize
        # store the dfa json in a file
        with open(file_path, "w") as f:
            json.dump(self.dfa_json, f, indent=4)

    def visualize_dfa(self, path="./dfa"):
        """_summary_
        Args:
            path (str, optional): Path where the gv file will be stores. Defaults to "./dfa.gv".
        """
        graph_visualize = GraphVisualize()
        if graph_visualize.graph_visualize(path, self.dfa_json):
            print(f"Visualization of the DFA is saved in {path}")
        else:
            print("Error: Visualization DFA failed")
        del graph_visualize

    def minimize_dfa(self, inputs) -> DFA:
        """_summary_
        This function is used to minimize the DFA
        Args:
            inputs (list): list of inputs of the NFA (Terminals in Regex)
        Returns:
            DFA: minimized DFA object
        """
        # the current state of the DFA
        pi = []
        # list of accepting states of the DFA
        accept = self._dfa.accept
        if accept:
            # add the accepting states to the current state of the DFA
            pi.append(set(accept))
        # list of non accepting states of the DFA
        non_accept = self._dfa.non_accept
        if non_accept:
            # add the non accepting states to the current state of the DFA
            pi.append(set(non_accept))
        print("Initial Partition: ", pi)
        # flag to tell if there is a change in the partition
        change = True
        while change:
            change = False
            # create a new partition
            new_pi = []
            for group in pi:
                if len(group) > 1:
                    # the splited states in the group
                    # key is the transition table for the state
                    splitted_states = {}
                    for state in group:
                        # transion table for each state
                        state_transition_table = {}
                        for input_ in inputs:
                            for transition in self._dfa.transitions:
                                if (
                                    transition.from_ == state
                                    and input_ in transition.characters
                                ):
                                    if input_ not in state_transition_table:
                                        state_transition_table[input_] = set()
                                    state_transition_table[input_].add(transition.to_)
                        str_temp = str(state_transition_table)
                        print("Transition Table: ", str_temp)
                        if str_temp not in splitted_states:
                            splitted_states[str_temp] = set()
                        splitted_states[str_temp].add(state)
                    # print("Splitted States: ", splitted_states)
                    # if there is more than one transition table for the group
                    if len(splitted_states) > 1:
                        # change is True to check on the new partitions
                        change = True
                        print("Splitted States: ", splitted_states)
                        # add the splited states to the new partition
                        for value in splitted_states.values():
                            new_pi.append(set(value))
                    else:
                        # if there is only one transition table for the group
                        # add the group to the new partition
                        new_pi.append(group)
                else:
                    # if the group contains only one state
                    # add the group to the new partition
                    new_pi.append(group)
            # if there is a change in the partition
            if change:
                pi = new_pi
                # print("New Partition: ", pi)
        # list of states of the minimized DFA
        min_states = []
        # list of transitions of the minimized DFA
        min_transitions = []
        # list of accepting states of the minimized DFA
        min_accept = []
        # list of non accepting states of the minimized DFA
        min_non_accept = []
        # start state of the minimized DFA
        start = None
        print("Final Partition: ", pi)
        # create the state sof the minimized DFA
        for i, group in enumerate(pi):
            state = DfaState(name=f"S{i}")
            min_states.append(state)
            if group.intersection(self._dfa.accept):
                min_accept.append(state)
            else:
                min_non_accept.append(state)
            # get the start state of the minimized DFA
            if self._dfa.start in group:
                start = state
        # create the transitions of the minimized DFA
        for transition in self._dfa.transitions:
            for i, group in enumerate(pi):
                if transition.from_ in group:
                    # note that the number of groups in pi is the number of states in min_states so each group maps to a state
                    from_ = min_states[i]
                if transition.to_ in group:
                    to_ = min_states[i]
            min_transitions.append(
                DfaEdge(from_=from_, to_=to_, characters=transition.characters)
            )
        # create a DFA object for the minimized DFA
        minimized_dfa = DFA(
            start=start,
            accept=min_accept,
            non_accept=min_non_accept,
            states=min_states,
            transitions=min_transitions,
        )
        return minimized_dfa


### Functions

In [8]:
def json_to_nfa(json_path: str) -> NFA:
    """
    This function is used to convert the JSON data to NFA
    params:
        json_path: str: Path to the JSON file
    return:
        NFA: NFA object
    """
    # read the JSON file
    with open(json_path, "r") as file:
        json_data = json.load(file)
    # create the NFA object
    nfa = NFA(None, None, [], [])
    # create the states
    states = []
    acceping_index = 0
    for state in json_data:
        if state == "startingState":
            continue
        if json_data[state]["isTerminatingState"] == True:
            acceping_index = int(state[1:])
        states.append(DfaState(name=state))
    nfa.states = states
    # set the start state becuase the name of the starting states is the index of it inside the states array
    nfa.start = states[int(json_data["startingState"][1:])]
    # set the accept state
    nfa.accept = states[acceping_index]
    # create the transitions
    transitions = []
    for state in json_data:
        if state == "startingState":
            continue
        for transition in json_data[state]:
            if transition == "isTerminatingState":
                continue

            # check if the transition is epsilon or not
            if transition.startswith("epsilon"):
                transitions.append(
                    DfaEdge(
                        states[int(state[1:])],
                        states[int(json_data[state][transition][1:])],
                        {EPSILON},
                    )
                )
            else:
                transitions.append(
                    DfaEdge(
                        states[int(state[1:])],
                        states[int(json_data[state][transition][1:])],
                        {transition},
                    )
                )
    nfa.transitions = transitions
    return nfa

In [9]:
# getting the inputs from the tansitions in the nfa
def get_inputs(transitions):
    """_summary_
    This function is used to get the inputs from the transitions in the NFA
    Args:
        transitions (List): List of transitions in the NFA (List of Edges)
    Returns:
        List: List of unique inputs in the NFA
    """
    inputs = []
    for transition in transitions:
        temp_in = list(transition.characters)[0]
        if temp_in not in inputs and temp_in != EPSILON:
            inputs.append(temp_in)
    return inputs

### Testing

In [10]:
input_regex = input("Enter the regular expression: ")
# input_regex = "[a-z]*[0-9]+"
nfa = NFA_CLASS(input_regex)
deserialized_nfa = json_to_nfa("./nfa.json")
inputs = get_inputs(deserialized_nfa.transitions)
dfa = DFA_CLASS()
dfa.subset_construction(deserialized_nfa, inputs)
dfa.rename_dfa_states()
dfa.dfa_to_json("dfa.json")
dfa.visualize_dfa()
# dfa.print_dfa()
minimized_dfa = dfa.minimize_dfa(inputs)
minimized_dfa_obj = DFA_CLASS(minimized_dfa,inputs)
minimized_dfa_obj.print_dfa()
minimized_dfa_obj.dfa_to_json("minimized_dfa.json")
minimized_dfa_obj.visualize_dfa(path="minimized_dfa")

Enter the regular expression: as
Visualization of the NFA is saved in nfa
Visualization of the DFA is saved in ./dfa
Initial Partition:  [{DfaState(name='S2')}, {DfaState(name='S0'), DfaState(name='S1')}]
Transition Table:  {'a': {DfaState(name='S1')}}
Transition Table:  {'s': {DfaState(name='S2')}}
Splitted States:  {"{'a': {DfaState(name='S1')}}": {DfaState(name='S0')}, "{'s': {DfaState(name='S2')}}": {DfaState(name='S1')}}
Final Partition:  [{DfaState(name='S2')}, {DfaState(name='S0')}, {DfaState(name='S1')}]
Start State:  S1
Accepting States:  ['S0']
Non Accepting States:  ['S1', 'S2']
States:  ['S0', 'S1', 'S2']
Transitions: 
From: S1, To: S2, Characters: {'a'}
From: S2, To: S0, Characters: {'s'}
Visualization of the DFA is saved in minimized_dfa
